In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [2]:
import numpy as np

a = np.arange(51)

a[-50:]


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

In [3]:
import tensorflow as tf

tf.get_logger().setLevel(tf._logging.ERROR)
print(tf.config.get_visible_devices())

from keras.layers import (
    Layer,
    MultiHeadAttention,
    Dense,
    LayerNormalization,
    Dropout,
    Embedding,
    Input,
    TextVectorization,
    Flatten,
    LeakyReLU,
)
from keras import Model, losses, Sequential, callbacks, activations, optimizers, utils
import tensorflow as tf
from typing import Literal
import numpy as np
import re
from tensorflow_addons.layers import InstanceNormalization
import string

from keras_nlp.layers import TransformerDecoder, TransformerEncoder, TokenAndPositionEmbedding

class SaveModel(tf.keras.callbacks.Callback):
    def __init__(self, path: str, **kwargs):
        super().__init__(**kwargs)
        self.path = path
    
    def on_epoch_end(self, epoch, logs=None):
        self.model.save(self.path, save_format="tf")


class TextGenerator(callbacks.Callback):
    def __init__(
        self,
        seed_text: str,
        next_words: int,
        max_sequence_len: int,
        vectorize_layer: TextVectorization,
        top_k=10,
        print_every=1,
        model=None,
    ):
        self.seed_text = seed_text
        self.next_words = next_words
        self.max_sequence_len = max_sequence_len
        self.vectorize_layer = vectorize_layer
        if model is not None:
            self.model: Model = model
        self.print_every = print_every
        self.k = top_k

        vocab = vectorize_layer.get_vocabulary()
        self.int2word = {i: word for i, word in enumerate(vocab)}
        self.word2int = dict(zip(self.int2word.values(), self.int2word.keys()))

    def preprocess(self, content: str) -> str:
        to_left: str = r" A-Za-ząćęłńóśźż\-.,?!:;()\n"
        content = re.sub(f"[^{to_left}]+", "", content).lower()
        # content = re.sub(f"([{string.punctuation}])", r" \1", content)
        content = re.sub("\n+", " \n ", content)
        content = re.sub(" +", " ", content)
        return content

    def sample_from(self, logits: np.ndarray) -> np.ndarray:
        indices = logits.argpartition(-self.k)[-self.k :].astype("int32")
        logits = logits[indices]

        preds = activations.softmax(tf.expand_dims(logits, 0))
        preds = np.array(preds[0]).astype("float32")
        return np.random.choice(indices, p=preds)

    def generate_text(self) -> str:
        start_tokens = self.preprocess(self.seed_text).split(" ")
        tokens_generated = []
        while len(tokens_generated) <= self.next_words:

            start_tokens = start_tokens[-self.max_sequence_len :]

            x = []
            for tok in start_tokens:
                if tok in self.word2int.keys():
                    x.append(self.word2int[tok])
            
            x = x[:-1]
            x = utils.pad_sequences(
                np.array(x)[np.newaxis], maxlen=self.max_sequence_len, padding="post"
            )
            y = x[1:]
            y = utils.pad_sequences(
                np.array(y)[np.newaxis], maxlen=self.max_sequence_len, padding="post"
            )


            pred = self.model.predict_on_batch([x, y])[0]
            idx = min(len(start_tokens) - 1, self.max_sequence_len - 1)

            sample_token = self.sample_from(pred[idx] if len(pred.shape) == 2 else pred)
            tokens_generated.append(sample_token)
            start_tokens.append(self.int2word[sample_token])

        token_to_word = []
        for tok in tokens_generated:
            try:
                word = self.int2word[tok]
                token_to_word.append(word)
            except:
                token_to_word.append("")
        txt = self.seed_text + " " + " ".join(token_to_word)
        txt = re.sub(r"\s([" + f"${string.punctuation}" + r"](?:\s|$))", r"\1", txt)
        return txt

    def on_epoch_begin(self, epoch: int, logs=None):
        if (epoch + 1) % self.print_every != 0:
            return
        txt = self.generate_text()
        print(f"Epoch: {epoch}; Generated text:\n{txt}\n")

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Using TensorFlow backend


/home/adrian/anaconda3/envs/tf_311/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:
import tensorflow as tf
from keras.layers import TextVectorization
from glob import glob
from typing import List, Literal
import string
import re
import numpy as np
from keras import utils
import json


fnames = list(glob("../texts/books-raw/*")) + list(glob("../texts/bajki-extend/*"))


def create_dataset(
    filenames: List[str],
    output_sequence_length: int = 50,
    max_tokens: int = 50_000,
    min_output_sequence_length: int = 5,
):
    to_left: str = r" A-Za-ząćęłńóśźż\-.,?!:;()\n"

    def clear_dataset(input_str: str) -> str:
        input_str = tf.strings.lower(input_str)
        input_str = tf.strings.regex_replace(input_str, f"[^{to_left}]+", "")
        # input_str = tf.strings.regex_replace(
        #     input_str, f"([{string.punctuation}])", r" \1"
        # )
        input_str = tf.strings.regex_replace(input_str, "\n+", " \n ")
        input_str = tf.strings.regex_replace(input_str, " +", " ")
        return input_str

    ds: tf.data.Dataset = tf.data.TextLineDataset(filenames)
    ds = ds.map(lambda x: clear_dataset(x))
    ds = ds.filter(lambda x: tf.strings.length(x) > 50)
    ds = ds.batch(512)
    ds = ds.prefetch(tf.data.AUTOTUNE)

    vectorize_layer = TextVectorization(
        standardize=None,
        max_tokens=max_tokens,
        output_mode="int",
        output_sequence_length=output_sequence_length,
    )
    vectorize_layer.adapt(ds)
    vocab = vectorize_layer.get_vocabulary()
    _word2int = {word: i for i, word in enumerate(vocab)}
    _int2word = dict(zip(_word2int.values(), _word2int.keys()))

    def data_generator(data: List[int], max_len: int):
        while True:
            x, y, z = [], [], []
            for _ in range(1024):
                _max_len = np.random.randint(min_output_sequence_length, max_len + 2)
                index = np.random.randint(len(data) - _max_len - 2)
                x.append(data[index : index + _max_len])
                y.append(data[index + 1 : index + _max_len + 1])
                z.append(data[index + 2 : index + _max_len + 2])

            x = utils.pad_sequences(x, max_len, padding="post")
            y = utils.pad_sequences(y, max_len, padding="post")
            z = utils.pad_sequences(z, max_len, padding="post")
            for _x, _y, _z in zip(x, y, z):
                yield (_x, _y), _z

    data = ""
    for filename in filenames:
        with open(filename, "r", encoding="utf8") as f:
            content = re.sub(f"[^{to_left}]+", "", f.read()).lower()
            # content = re.sub(f"([{string.punctuation}])", r" \1", content)
            content = re.sub("\n+", " \n ", content)
            content = re.sub(" +", " ", content)
            data += content

    data_int = []
    for key in data.split(" "):
        if key in _word2int.keys():
            data_int.append(_word2int[key])

    ds = tf.data.Dataset.from_generator(
        lambda: data_generator(data_int, output_sequence_length),
        output_signature=(
            (tf.TensorSpec(shape=(output_sequence_length,), dtype=tf.int32), tf.TensorSpec(shape=(output_sequence_length,), dtype=tf.int32)),
            tf.TensorSpec(shape=(output_sequence_length,), dtype=tf.int32),
        ),
    )
    ds = ds.prefetch(tf.data.AUTOTUNE)

    return ds, _word2int, _int2word, vectorize_layer


def create_model(max_sequence_len: int, total_words: int) -> Model:
    embed_dim = 256
    num_heads = 4

    inputs1 = Input(shape=(max_sequence_len,))
    inputs2 = Input(shape=(max_sequence_len,))
    x = TokenAndPositionEmbedding(max_sequence_len, total_words, embed_dim, mask_zero=True)(inputs1)
    y = TokenAndPositionEmbedding(max_sequence_len, total_words, embed_dim, mask_zero=True)(inputs2)

    x = TransformerEncoder(embed_dim, num_heads, 0.1)(x)
    y = TransformerDecoder(embed_dim, num_heads, 0.1)(y, x)


    outputs = Dense(total_words)(y)
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(
        loss=losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=optimizers.Adam(1e-3),
    )
    return model

In [5]:

output_sequence_length = 50
min_output_sequence_length = 5
max_tokens = 100_000
epochs = 30
ds, word2int, int2word, vectorize_layer = create_dataset(
    fnames, output_sequence_length, max_tokens, min_output_sequence_length
)

seed = "Kiedy księżyc wzeszedł, wziął Jaś siostrzyczkę za rękę i poszedł z nią śladem kamyków, które błyszczały w świetle księżycowym jak nowiutkie pieniążki i pokazywały im drogę. Szli całą noc, a gdy dzień nastał, doszli do domu ojca.".lower()

model = create_model(output_sequence_length, max_tokens)
model.summary()
model.fit(
    ds.shuffle(256).batch(128).prefetch(tf.data.AUTOTUNE),
    verbose=1,
    epochs=epochs,
    steps_per_epoch=1500,
    callbacks=[
        TextGenerator(seed, 60, output_sequence_length, vectorize_layer, 5),
        SaveModel("../transformer_models/model_best_2.tf")
        # tf.keras.callbacks.ModelCheckpoint(
        #     "../transformer_models/model_best_2.h5",
        #     monitor="loss",
        #     save_best_only=True,
        #     save_weights_only=False,
        # ),
    ],
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 token_and_position_embedding (  (None, 50, 256)     25612800    ['input_1[0][0]']                
 TokenAndPositionEmbedding)                                                                       
                                                                                                  
 token_and_position_embedding_1  (None, 50, 256)     25612800    ['input_2[0][0]']            

1500/1500 [==============================] - 284s 188ms/step - loss: 6.9572
Epoch: 1; Generated text:
kiedy księżyc wzeszedł, wziął jaś siostrzyczkę za rękę i poszedł z nią śladem kamyków, które błyszczały w świetle księżycowym jak nowiutkie pieniążki i pokazywały im drogę. szli całą noc, a gdy dzień nastał, doszli do domu ojca. i do w się w w się się w w w w i to to i i w w w i w w i w w w w i w w w w w i do w w w w i i i i w w w w w w w w w w w w w i i i w

Epoch 2/30
1500/1500 [==============================] - ETA: 0s - loss: 6.2838

1500/1500 [==============================] - 284s 189ms/step - loss: 6.2838
Epoch: 2; Generated text:
kiedy księżyc wzeszedł, wziął jaś siostrzyczkę za rękę i poszedł z nią śladem kamyków, które błyszczały w świetle księżycowym jak nowiutkie pieniążki i pokazywały im drogę. szli całą noc, a gdy dzień nastał, doszli do domu ojca. w się na się się w w i się na z się na na w się w na i w w na i w na w na i i na i się w i i się i pod gdy i w w i w w w się i się na w w i na w z w i i i i

Epoch 3/30
1500/1500 [==============================] - ETA: 0s - loss: 6.1477

1500/1500 [==============================] - 284s 190ms/step - loss: 6.1477
Epoch: 3; Generated text:
kiedy księżyc wzeszedł, wziął jaś siostrzyczkę za rękę i poszedł z nią śladem kamyków, które błyszczały w świetle księżycowym jak nowiutkie pieniążki i pokazywały im drogę. szli całą noc, a gdy dzień nastał, doszli do domu ojca. się i w na na i się i i się się z z w i w z i i w i w z i na i się w z z w i z się i i i i się w z się się i się i na i na się się i i na na i co i i co i

Epoch 4/30
1500/1500 [==============================] - ETA: 0s - loss: 6.0617

1500/1500 [==============================] - 287s 191ms/step - loss: 6.0617
Epoch: 4; Generated text:
kiedy księżyc wzeszedł, wziął jaś siostrzyczkę za rękę i poszedł z nią śladem kamyków, które błyszczały w świetle księżycowym jak nowiutkie pieniążki i pokazywały im drogę. szli całą noc, a gdy dzień nastał, doszli do domu ojca. i na na się się z się w na na i na na i na i w w i z się w i z i na i na z w i na i w z z i i na z i i na i na i i i na do i na i na na na na i i i i

Epoch 5/30
1500/1500 [==============================] - ETA: 0s - loss: 5.9951

1500/1500 [==============================] - 278s 185ms/step - loss: 5.9951
Epoch: 5; Generated text:
kiedy księżyc wzeszedł, wziął jaś siostrzyczkę za rękę i poszedł z nią śladem kamyków, które błyszczały w świetle księżycowym jak nowiutkie pieniążki i pokazywały im drogę. szli całą noc, a gdy dzień nastał, doszli do domu ojca. w i i na w i z i z i się się z z i w z i i z z na i z na na z i a i i i z na z z z z na z i z i i i z i na z i i a ze i na ze i na na z na

Epoch 6/30
 657/1500 [============>.................] - ETA: 2:35 - loss: 5.9458

KeyboardInterrupt: 

In [ ]:
fnames2 = list(glob("../texts/bajki-extend/*"))
ds2, _, _, _ = create_dataset(fnames2)

model.fit(
    ds.batch(128),
    verbose=1,
    epochs=10,
    steps_per_epoch=1500,
    callbacks=[
        TextGenerator(seed, 60, output_sequence_length, vectorize_layer, 5),
        tf.keras.callbacks.ModelCheckpoint(
            "../transformer_models/model_best_3.h5",
            monitor="loss",
            save_best_only=True,
            save_weights_only=False,
        ),
    ],
)

In [ ]:
seed = "spójrz , jak pięknie kwitną dokoła kwiatki , dlaczego nie patrzysz na nie ?".lower()
for i in [1, 2, 4, 8, 10, 20]:
    txt = TextGenerator(seed, 60, output_sequence_length, vectorize_layer, i, model=model).generate_text(),
    print(txt)
    # with open(f'../generated_texts/transformer/text_{i}', 'w') as f:
    #     f.write(f'Seed: {seed}\n')
    #     f.write(txt)